# Aprendizagem por reforço: jogo da velha

 Número de posições possíveis com pelo menos duas casas livres = $3^7 \times 72 = 157.464$ (número de possibilidades para um tabuleiro com $7$ casas, e $3$ possibilidades por casa - O, X e vazio - vezes o número de maneiras possíveis de escolher as duas casas livres)
 
## Modelo Beta-Binomial

Para uma certa posição $x$, com $k(x)$ casas livres, tenho $n(x)$ ações possíveis. A partir de agora, vou ignorar o $x$, mantendo sempre em mente que teremos um modelo desses abaixo para cada uma das $157.464$ posições.

Para cada uma das $k$ ações possíveis, defino uma probabilidade $\pi(i)$: a probabilidade de vitória caso o jogador escolha a a ação $i$ ($i=1,...k$)

Observo $n_i$ jogos começando da mesma posição, e escolho sempre a ação $i$. Conto o número de vitórias $v_i$. Repito isso para todas as $k$ posições; ou seja, observo $(v_i, n_i)$ para $i=1,...,k$.  

Supondo que conheço as probabilidades $\pi(i)$, a probabilidade associada ao número de jogos e vitórias observadas (a verossimilhança) é uma binomial:

$P(v_i | n_i, \pi(i)) \propto \pi(i)^{v_i}(1-\pi(i))^{n_i-v_i}$

Considerando agora todas as posições, e supondo que os resultados dos jogos são independentes (condicionalmente a $\pi$), temos a verossimilhança conjunta de todas as posições:

$P(\mathbf{v} | \mathbf{n}, \mathbf{\pi}) \propto \prod_{i=1}^k \pi(i)^{v_i}(1-\pi(i))^{n_i-v_i}$

Como não conheço $\pi$, vou modelar minha ignorância usando uma distribuição de probabilidade para esses valores (uma priori). Por questões matemáticas, escolho uma distribuição Beta para cada possível ação:

$P(\pi(i)) \propto \pi(i)^{\alpha_i - 1}(1-\pi(i))^{\beta_i-1}$

Onde $\alpha_i$ e $\beta_i$ são hiperparâmetros que vamos precisar escolher (em lugar de aprender seus valores). 

## Aprendizagem e o teorema de Bayes

Eu começo então dizendo que a probabilidade de vitória do movimento $i$ tem distribuição a priori Beta com hiperparâmetros $\alpha_i$ e $\beta_i$; jogo uma vez e escolho o movimento $i$. Como incorporo essa nova informação?

Pelo teorema de Bayes, e usando a propriedade de conjugação entre priori Beta e verossimilhança binomial, sei que a nova distribuição de $\pi(i)$ é de novo uma Beta, mas agora com parâmetros $(\alpha_i + 1, \beta_i)$ em caso de vitória, e $(\alpha_i, \beta_i+1)$ em caso de derrota. 

Portanto a equação de aprendizagem desse modelo, para cada posição $x$ e cada possível movimento $i_x$, é:

$\begin{align}
&\alpha_i(t+1) = \alpha_i(t) + r_i(t) \\
&\beta_i(t+1) = \beta_i(t) + (!-r_i(t))
\end{align}$

onde $r_i(t) = 1$ no caso de vitória, e $0$ caso contrário.

Observação: agora temos uma interpretação para os hiperparâmetros $\alpha$ e $\beta$; eles representam quantas vitórias e quantas derrotas o robô viu até o momento em que começa a nova rodada de aprendizagem.

## Decisão

O robô chegou numa posição $x$ com diversas ações possíveis. Ele tem uma distribuição de probabilidade para a probabilidade de vitória (a probabilidade de uma probabilidade...) de cada lance. O que fazer?

1. Olhar o valor esperado de probabilidade de vitória para cada movimento; escolher o movimento com maior valor esperado.
2. Sortear uma probabilidade de vitória para cada movimento, conforme a distribuição atual; escolher o movimento com maior valor sorteado

In [3]:
%matplotlib tk
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

class owg:
    '''
    Classe owg: define o tabuleiro e as regras do jogo, aceita movimentos e mantém a posição atual do jogo
    na memória.
    
    '''
    def __init__(self):
        self.state = np.empty(shape = (3, 3))
        self.cur = None
        self.state[:] = np.nan
        self.__convert_state()
        self.starter = 0

    def reset(self):
        '''
        Reseta o tabuleiro, para começar novo jogo
        '''
        self.state = np.empty(shape = (3, 3))
        self.state[:] = np.nan
        self.__convert_state()
        self.starter = 1 - self.starter
        self.cur = None
        
    def __check_move(self, i, j):
        return np.isnan(self.state[i,j])
    
    def play(self, jogador, movimento):
        '''
        Jogador faz o movimento
        
        
        @args
        
        jogador -- 0, 1 ou None. Se None, o tabuleiro realiza a jogada para o próximo jogador. 
                        0 ou 1 para representar um jogador específico. Em tese esse parâmetro não preciso ser 
                        utilizado a menos que você queira manter um controle de erro sobre quem está jogando 
                        naquela vez.
        movimento -- uma tupla (linha, coluna), linha \in [0,1,2] e coluna \in [0,1,2]. O jogador atual marca
                        a posição linha, coluna.
                        
        
        @returns
        
        Retorna True se bem-sucedida
        
        '''
        i = movimento[0]
        j = movimento[1]
        
        if not self.__check_move(i,j):
            #raise ValueError("Erro! movimento ({},{}) não permitido".format(i,j))
            print("Erro! movimento ({},{}) não permitido".format(i,j))
            return False
        
        if jogador is None:
            if self.cur is None:
                self.cur = 0
            jogador = self.cur
            
        if jogador not in [0,1]:
            raise ValueError("Erro! jogador deve ser 0 ou 1")
        
        if self.cur is None:
            self.cur = jogador
        else:
            if self.cur != jogador:
                raise ValueError("Erro! Não é a vez do jogador {}".format(player))
        
        self.state[i,j] = jogador
        self.__convert_state()
        self.cur = 1 - self.cur
        return True
        
    def check_result(self):
        '''
        Método para verificar se o jogo acabou, usando a soma das colunas / linhas.        
        
        @returns
        
        resultado, motivo -- resultado é None se o jogo não acabou ainda; 1 se o jogador 1 ganhou;
                                -1 se o jogador 0 ganhou; 0 se foi empate.
                             motivo é None se o jogo não acabou ainda, ou se foi empate. Caso contrário, dá
                             as coordenadas da linha, coluna ou diagonal que causou o fim do jogo
                                
        ''' 
        colsum = self.state.sum(axis = 1)
        if np.any(colsum == 3):
            # 1 ganhou
            i = int(np.where(colsum == 3)[0])
            return 1, (3,i)
        elif np.any(colsum == 0):
            # 0 ganhou
            i = int(np.where(colsum == 0)[0])
            return -1, (3,i)
        rowsum = self.state.sum(axis= 0)
        if np.any(rowsum == 3):
            # 1 ganhou
            i = int(np.where(rowsum == 3)[0])
            return 1, (i,3)
        elif np.any(rowsum == 0):
            # 0 ganhou
            i = int(np.where(rowsum == 0)[0])
            return -1, (i,3)
        # Diagonal
        diag = self.state[0,0] + self.state[1,1] + self.state[2,2]
        if diag == 0:
            return -1, (-3,-3)
        elif diag == 3:
            return 1, (-3,-3)
        
        # Segunda diagonal
        diag = self.state[0,2] + self.state[1,1] + self.state[2,0]
        if diag == 0:
            return -1, (3,3)
        elif diag == 3:
            return 1, (3, 3)
        
        
        if np.any(np.isnan(self.state)):
            # ainda não acabou
            return None, None
        else:
            # empatou
            return 0, None
        
    def __convert_state(self):
        '''
        Interna, para converter o estado do tabuleiro num inteiro em base 3
        0 = O, 1 = X, None = empty position
        '''

        istate = 0
        tmpstate = self.state.copy()
        tmpstate[np.isnan(tmpstate)] = 2
        tmpstate = tmpstate.flatten().astype(int)

        sstate = np.array2string(tmpstate, separator = "", prefix= "")[1:-1]
            
        self.istate = int(sstate, 3)
        self.sstate = sstate
        
    def start(self):
        '''
        Inicia a janela com o tabuleiro para jogo entre dois humanos.
        
        '''
        
        # Número de pontos para desenhar "X"       
        N = 2000
        
        # Número de pontos para desenhar "O"
        Nbola = 10000
        
        # Gera conjunto de pontos para as figuras
        # X
        xx = np.random.normal(loc = 0, scale = 0.05, size = N)
        yx1 = 2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 
        yx2 = -2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 

        # Bola
        xb = np.random.normal(loc = 0, scale = 0.05, size = Nbola)
        yb = np.random.normal(scale = 0.1, size = len(xb)) 

        # Reseta o tabuleiro atual
        self.reset()
        
        # Método para capturar os eventos de clique no tabuleiro
        def onclick(self, event):
            ix, iy = event.xdata, event.ydata
            cur = tab.cur

            r, tipo = self.check_result()

            if r is not None:
                # Jogo acabou, redesenha
                plt.cla()
                plt.vlines(x = 1/2, ymin = 0, ymax = 3)
                plt.vlines(x = 1, ymin = 0, ymax = 3)
                plt.hlines(y = 1, xmin = 0, xmax = 3/2)
                plt.hlines(y = 2, xmin = 0, xmax = 3/2)
                plt.axis('off')
                self.reset()
            else:
                # Jogo não acabou, desenha o símbolo correspondente ao jogador atual no quadrado clicado
                if ix > 0.0 and ix < 1.5:
                    if ix <= 0.5:
                        i = 0
                    elif ix <= 1.:
                        i = 1
                    else:
                        i = 2
                else:
                    i = None

                if iy > 0.0 and iy <= 3:
                    if iy <= 1.:
                        j = 0
                    elif iy <= 2:
                        j = 1
                    else:
                        j = 2
                else:
                    j = None

                if i is not None and j is not None:
                    # Se o clique foi num quadrado válido do tabuleiro
                    
                    # Executa o movimento
                    res = self.play(None, (i,j))

                    if res:
                        # Centro do quadrado
                        xc = i/2 + .235
                        yc = 0.5 + j 
                        if tab.cur:
                            # Jogador atual é o X
                            x = xx + xc
                            y = yx1 + yc
                            y2 = yx2 + yc                    
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                            plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                        else:
                            # Jogador atual é o O
                            x = xb + xc
                            y = yb + yc
                            z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                            x = [zz[0] for zz in z]
                            y = [zz[1] for zz in z]
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')


                    # Verifica se o jogo acabou após esse movimento
                    r, tipo = self.check_result()

                    if r is not None:
                        if r == 0:
                            print("Empate")
                        elif r == 1:
                            print("X ganhou")
                            # Desenha no tabuleiro a reta que indica o motivo da vitória
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        
                        elif r == -1:
                            print("O ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        

            plt.show()


        fig = plt.figure(figsize = (10, 10))

        plt.vlines(x = 1/2, ymin = 0, ymax = 3)
        plt.vlines(x = 1, ymin = 0, ymax = 3)
        plt.hlines(y = 1, xmin = 0, xmax = 3/2)
        plt.hlines(y = 2, xmin = 0, xmax = 3/2)
        plt.axis('off')

        cid = fig.canvas.mpl_connect('button_press_event', lambda l:onclick(self, l))


        plt.show()        
        
        
    def start(self, p1):
        '''
        Tabuleiro para humano x computador (p1)
        
        @args
        
        p1 -- um objeto da classe owg_player
        '''
        
        p1.reset()
        
        # Número de pontos para desenhar o X
        N = 2000
        
        # Número de pontos para desenharo O
        Nbola = 10000
        
        # Gera conjunto de pontos para as figuras
        # X
        xx = np.random.normal(loc = 0, scale = 0.05, size = N)
        yx1 = 2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 
        yx2 = -2*xx + np.random.normal(scale = 0.1/3, size = len(xx)) 

        # Bola
        xb = np.random.normal(loc = 0, scale = 0.05, size = Nbola)
        yb = np.random.normal(scale = 0.1, size = len(xb)) 

        self.reset()
        
        # Função para capturar o evento de clique no tabuleiro
        def onclick(self, event):
            ix, iy = event.xdata, event.ydata

            # Checa o resultado
            r, tipo = self.check_result()

            if r is not None:
                # Jogo acabou, redesenha o tabuleiro
                plt.cla()
                plt.vlines(x = 1/2, ymin = 0, ymax = 3)
                plt.vlines(x = 1, ymin = 0, ymax = 3)
                plt.hlines(y = 1, xmin = 0, xmax = 3/2)
                plt.hlines(y = 2, xmin = 0, xmax = 3/2)
                plt.axis('off')
                self.reset()
                self.cur = self.starter
                p1.reset()
                
                # Verifica quem começa o próximo jogo
                if self.starter == 1:
                    # Computador joga
                    movimento = p1.joga()

                    # Processa o movimento proposto
                    if movimento is not None:
                        res = self.play(None, movimento)
                        i = movimento[0]
                        j = movimento[1]
                        if res:
                            # Centro do quadrado
                            xc = i/2 + .235
                            yc = 0.5 + j 
                            if self.cur:
                                x = xx + xc
                                y = yx1 + yc
                                y2 = yx2 + yc                    
                                plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                                plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                            else:
                                x = xb + xc
                                y = yb + yc
                                z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                                x = [zz[0] for zz in z]
                                y = [zz[1] for zz in z]
                                plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')                
            else:
                # Jogo não acabou
                # Processa as coordenadas do clique, se for movimento válido realiza o movimento
                if ix > 0.0 and ix < 1.5:
                    if ix <= 0.5:
                        i = 0
                    elif ix <= 1.:
                        i = 1
                    else:
                        i = 2
                else:
                    i = None

                if iy > 0.0 and iy <= 3:
                    if iy <= 1.:
                        j = 0
                    elif iy <= 2:
                        j = 1
                    else:
                        j = 2
                else:
                    j = None

                if i is not None and j is not None:
                    res = self.play(None, (i,j))
                    if res:
                        # Centro do quadrado
                        xc = i/2 + .235
                        yc = 0.5 + j 
                        if self.cur:
                            x = xx + xc
                            y = yx1 + yc
                            y2 = yx2 + yc                    
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                            plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                        else:
                            x = xb + xc
                            y = yb + yc
                            z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                            x = [zz[0] for zz in z]
                            y = [zz[1] for zz in z]
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')
                        

                    # Verifica o resultado após o movimento
                    r, tipo = self.check_result()

                    if r is not None:
                        # Acabou
                        if r == 0:
                            print("Empate")
                        elif r == 1:
                            print("X ganhou")
                            # Desenha a linha que indica o motivo da vitória
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        
                        elif r == -1:
                            print("O ganhou")
                            # Desenha a linha que indica o motivo da vitória
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        

                # Comunica o movimento atual ao robô
                p1.comunica((i,j))     
                # Robô joga
                movimento = p1.joga()

                # Processa o movimento
                if movimento is not None:
                    res = self.play(None, movimento)
                    i = movimento[0]
                    j = movimento[1]
                    if res:
                        # Centro do quadrado
                        xc = i/2 + .235
                        yc = 0.5 + j 
                        if self.cur:
                            x = xx + xc
                            y = yx1 + yc
                            y2 = yx2 + yc                    
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                            plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                        else:
                            x = xb + xc
                            y = yb + yc
                            z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                            x = [zz[0] for zz in z]
                            y = [zz[1] for zz in z]
                            plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')


                    # Verifica se o jogo acabou após movimento do robô
                    r, tipo = self.check_result()

                    if r is not None:
                        if r == 0:
                            print("Empate")
                        elif r == 1:
                            print("X ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0., 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                        
                        elif r == -1:
                            print("O ganhou")
                            if tipo[0] == 3 and tipo[1] != 3:
                                # Ganhou na coluna
                                col = tipo[1]
                                xr = np.repeat(0.25 + col*0.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[1] == 3 and tipo[0] != 3:
                                # Ganhou na linha
                                lin = tipo[0]
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.repeat(0.5 + lin, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            elif tipo[0] == 3:
                                # Ganhou na diagonal principal
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(3, 0, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)
                            else:
                                # Ganhou na diagonal secundária
                                xr = np.linspace(0, 1.5, 10)
                                yr = np.linspace(0, 3, 10)
                                plt.plot(xr, yr, '-', color = 'red', linewidth = 5)                                                        


            plt.show()


        fig = plt.figure(figsize = (10, 10))

        plt.vlines(x = 1/2, ymin = 0, ymax = 3)
        plt.vlines(x = 1, ymin = 0, ymax = 3)
        plt.hlines(y = 1, xmin = 0, xmax = 3/2)
        plt.hlines(y = 2, xmin = 0, xmax = 3/2)
        plt.axis('off')
        
        
        # Quem começa?
        if self.starter == 0:
            # Robô começa
            movimento = p1.joga()

            # Processa movimento
            if movimento is not None:
                res = self.play(None, movimento)
                i = movimento[0]
                j = movimento[1]
                if res:
                    # Centro do quadrado
                    xc = i/2 + .235
                    yc = 0.5 + j 
                    if self.cur:
                        x = xx + xc
                        y = yx1 + yc
                        y2 = yx2 + yc                    
                        plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')
                        plt.scatter(x, y2, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'blue')    
                    else:
                        x = xb + xc
                        y = yb + yc
                        z = [z for z in zip(x,y) if .25*(z[1]-yc)**2 + (z[0]-xc)**2 >= 0.008 + np.random.normal(scale = 0.008/5) ]
                        x = [zz[0] for zz in z]
                        y = [zz[1] for zz in z]
                        plt.scatter(x, y, s =  np.random.normal(loc = 2, scale = 0.1, size = len(x)), c = 'orange')

        cid = fig.canvas.mpl_connect('button_press_event', lambda l:onclick(self, l))


        plt.show()                

In [5]:
class owg_player:
    '''
    Classe para representar um jogador genérico. Implementa os métodos em comum.
    Os jogadores específicos com suas estratégias serão classes herdadas desta.
    '''
    def __init__(self):
        # board é um objeto owg
        # knowledge é um dicionário 'posicao' : [probabilidades], que atribui probabilidades a cada movimento na posição posicao
        # jogo é uma lista de pares ordenados (posição, ação) para guardar o histórico dos movimentos
        # O jogador sempre se considera internamente o jogador 1 (é irrelevante se ele é o X ou a O)
        self.knowledge = dict()
        self.board = owg()
        self.jogo = []
        
    def comunica(self, movimento, verbose = False):
        '''
        Jogador recebe informação sobre movimento do oponente, e altera seu estado interno.
        
        @args
        
        movimento -- dupla (i,j) com i a linha e j a coluna do movimento
        
        
        '''
        # Comunica o movimento do oponente
        self.board.play(0, movimento)
        if verbose:
            print("Comunica", movimento, self.board.state)


    def __inicializa(self, strpos):
        '''
        Método virtual, para ser sobrescrito pelos jogadores específicos
        '''
        pass
    
    def joga(self, verbose = False):
        '''
        Método virtual, para ser sobrescrito pelos jogadores específicos
        '''
        pass
    
    def reset(self):
        '''
        Reseta o estado do jogo atual
        '''
        self.board.reset()
        self.jogo = []

In [6]:
class jb (owg_player) : 
    ''' 
    Classe jb: é o jogador que não aprende; decide cada movimento aleatoriamente para sempre
    '''
    
    def __init__(self):
        self.nome = 'JB'
        owg_player.__init__(self)
        
    def __inicializa(self, strpos):
        '''
        Cria o vetor de probabilidade uniforme para as ações possíveis a partir da posição strpos.
        
        @args
        
        strpos -- uma string que representa a posição atual. A string tem nove posições, cada uma
                    correspondendo a uma célula do tabuleiro (da esquerda pra direita, de cima pra baixo).
                    '0' significa posição marcada pelo oponente, '1' significa posição marcada pelo próprio
                    jogador, e '2' significa posição vazia.
        '''
        # Cria o vetor de probabilidade uniforme para a dada posição
        
        # As ações possíveis são os quadrados vazios
        acoes = [i for i in range(len(strpos)) if strpos[i] == '2']
        
        # Cria as probabilidades da uniforme
        probs = [1/len(acoes)] * len(acoes)
        self.knowledge[strpos] = (acoes, probs)
    
    
    def joga(self, verbose = False):
        '''
        Dada a posição atual do tabuleiro, decide a ação. 
        '''
        
        # Primeiro verifica se jogo está rolando
        r, _ = self.board.check_result()
        if r is None:
            # Está rolando
            
            # Recupera string que representa o estado atual
            strpos = self.board.sstate
            if verbose:
                print(strpos)
            if strpos not in self.knowledge.keys():
                # Nunca viu essa posição
                self.__inicializa(strpos)
            # Obtém as ações possiveis e suas respectivas probabilidades de sucesso
            acoes, probs = self.knowledge[strpos]
            
            # Sorteia uma ação
            acao = np.random.choice(a = acoes, p = probs)
            
            # Constrói a dupla que representa o movimento
            movimento = (int(acao / 3), acao % 3)
            
            # Joga e armazena o movimento
            self.board.play(1, movimento)
            self.jogo.append((strpos, acao))
            
            return movimento
        else:
            return None

In [8]:
class cientista(owg_player):
    ''' 
    Cientista cético: usa o modelo probabilístico beta-binomial e o terema de Bayes para aprender 
        e atualizar as probabilidades.
        Sorteia a ação, conforme probabilidades de sucesso, para equilibrar exploration e exploitation.
    '''
    def __init__(self, a = 1, b = 1):
        '''
        @args 
        
        a, b -- números positivos, os parâmetros iniciais para cada priori Beta sobre as probabilidades de sucesso
        '''
        
        self.a = a
        self.b = b
        self.nome = 'Cientista'
        owg_player.__init__(self)
        
    def __inicializa(self, strpos):
        '''
        Cria o vetor de probabilidade uniforme para a dada posição, e cria a lista de alfas e betas
        ''' 
        acoes = [i for i in range(len(strpos)) if strpos[i] == '2']
        alfa = [self.a] * len(acoes)
        beta = [self.b] * len(acoes)
        self.knowledge[strpos] = (acoes, alfa, beta)
        
    def joga(self, verbose = False):
        '''
        Dada a posição atual do tabuleiro, decide a ação. Aprende com o resultado
        '''

        # Primeiro verifica se jogo está rolando
        r, _ = self.board.check_result()
        if r is None:
            # Está rolando
            # Recupera a string que representa a posição
            strpos = self.board.sstate
            
            if verbose:
                print(strpos)
            if strpos not in self.knowledge.keys():
                # Nunca viu essa posição
                # Inicializa da prior
                self.__inicializa(strpos)

            # Obtém as ações para aquela posição, e os respectivos parâmetros da Beta
            acoes, alfa, beta = self.knowledge[strpos]
            
            # Sorteia as probabilidades de sucesso da Beta
            probs = [np.random.beta(a = param[0], b = param[1]) for param in zip(alfa, beta)]
            
            # Sorteia a ação conforme as probabilidades de sucesso sorteadas no passo anterior
            acao = acoes[np.argmax(probs)]

            # Constrói o par que representa o movimento
            movimento = (int(acao / 3), acao % 3)

            # Joga e armazena
            self.board.play(1, movimento)
            self.jogo.append((strpos, acao))
            
            # Verifica se ganhou o jogo
            r, _ = self.board.check_result()
            if r is not None:
                # Terminou o jogo
                if r == 1:
                    # Ganhei!
                    # Distribui as recompensas, i.e., soma 1 no alfa correspondente a cada movimento
                    # executado na partida
                    for i in range(len(self.jogo)):
                        pos = self.jogo[i][0]
                        acao = self.jogo[i][1]
                        a, al, be = self.knowledge[pos]
                        al[a.index(acao)] += 1
                        self.knowledge[pos] = (a, al, be)

            return movimento
        else:
            if r == -1:
                # Perdi :-/
                # Distribui os castigos, i.e., soma 1 no beta correspondente a cada movimento
                # executado na partida
                for i in range(len(self.jogo)):
                    pos = self.jogo[i][0]
                    acao = self.jogo[i][1]
                    a, al, be = self.knowledge[pos]
                    be[a.index(acao)] += 1
                    self.knowledge[pos] = (a, al, be)
                    
            return None        
    

In [9]:
class cientista_sovina(owg_player):
    ''' 
    Cientista sovina: usa o modelo probabilístico para aprender e atualizar as probabilidades, mas 
    é ganacioso na hora de decidir a ação (sempre escolhe a que tem maior probabilidade esperada de sucesso)
    '''
    def __init__(self, a = 1, b = 1):
        '''
        @args
        
        a, b -- hiperparâmetros iniciais da priori Beta
        '''
        self.a = a
        self.b = b
        self.nome = 'Cientista sovina'
        owg_player.__init__(self)
        
    def __inicializa(self, strpos):
        ''' 
        Cria o vetor de probabilidade uniforme para a dada posição
        '''
        acoes = [i for i in range(len(strpos)) if strpos[i] == '2']
        alfa = [self.a] * len(acoes)
        beta = [self.b] * len(acoes)
        self.knowledge[strpos] = (acoes, alfa, beta)
        
    def joga(self, verbose = False):
        '''
        Dada a posição atual do tabuleiro, decide a ação. Aprende com o resultado.
        '''
        
        # Primeiro verifica se jogo está rolando
        r, _ = self.board.check_result()
        if r is None:
            # Está rolando
            
            # Recupera a string que representa a posição atual
            strpos = self.board.sstate
            
            if verbose:
                print(strpos)
            if strpos not in self.knowledge.keys():
                # Nunca viu essa posição
                self.__inicializa(strpos)

            # Recupera as ações possíveis e seus respectivos parâmetros
            acoes, alfa, beta = self.knowledge[strpos]
            
            # Calcula a probabilidade esperada de sucesso para cada posição
            probs = [param[0] / (param[0] + param[1]) for param in zip(alfa, beta)]
            # Escolhe a ação com maior prob esperada de sucesso
            acao = acoes[np.argmax(probs)]
            
            # Constrói a dupla que define a ação
            movimento = (int(acao / 3), acao % 3)

            # Joga e armazena
            self.board.play(1, movimento)
            self.jogo.append((strpos, acao))
            
            # Verifica se ganhou o jogo
            r, _ = self.board.check_result()
            if r is not None:
                if r == 1:
                    # Ganhei!
                    # Distribui as recompensas, i.e., soma 1 aos alfas correspondentes a cada movimento
                    # utilizado na partida
                    for i in range(len(self.jogo)):
                        pos = self.jogo[i][0]
                        acao = self.jogo[i][1]
                        a, al, be = self.knowledge[pos]
                        al[a.index(acao)] += 1
                        self.knowledge[pos] = (a, al, be)

            return movimento
        else:
            if r == -1:
                # Perdi :-/
                # Distribui os castigos, i.e., soma 1 aos betas correspondentes a cada movimento
                # utilizado na partida
                for i in range(len(self.jogo)):
                    pos = self.jogo[i][0]
                    acao = self.jogo[i][1]
                    a, al, be = self.knowledge[pos]
                    be[a.index(acao)] += 1
                    self.knowledge[pos] = (a, al, be)
                    
            return None      

In [ ]:
p1 = cientista()

In [ ]:
p2 = jb()

In [ ]:
p1.reset()
p2.reset()

In [ ]:
vitorias = 0
derrotas = 0
empates = 0
pempate = []
pjog1 = []
pjog2 = []
n = 1000000

for i in range(n):
    
    if i % 2 == 0:
        movimento = p1.joga()
        p2.comunica(movimento)
        res, _ = p2.board.check_result()
        while res is None:
            movimento = p2.joga()
            if movimento is not None:
                p1.comunica(movimento)
                movimento = p1.joga()           
                if movimento is not None:
                    p2.comunica(movimento)
            res, _ = p2.board.check_result()
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
    else:
        movimento = p2.joga()
        p1.comunica(movimento)
        res, _ = p1.board.check_result()
        while res is None:
            movimento = p1.joga()
            if movimento is not None:
                p2.comunica(movimento)
                movimento = p2.joga()          
                if movimento is not None:
                    p1.comunica(movimento)
            res, _ = p1.board.check_result()
        r, _ = p1.board.check_result()
        if r == 1:
            vitorias += 1
        elif r == -1:
            derrotas += 1
        else:
            empates += 1
        p1.reset()
        p2.reset()
        
    print("Jogo {}: P1 {:.2%}, P2 {:.2%}, Empate {:.2%}".format(i+1, vitorias / (i+1), derrotas / (i+1), empates / (i+1)))
    pempate.append(empates / (i+1))
    pjog1.append(vitorias / (i+1))
    pjog2.append(derrotas / (i+1))

In [ ]:
import pickle

with open('p1_1000000.pkl', 'wb') as arq:
    pickle.dump(p1, arq)

In [7]:
import pickle
with open('p1_1000000.pkl', 'rb') as arq:
    p1 = pickle.load(arq)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize = (20, 10))
plt.plot(pjog1, '-', label = p1.nome)
plt.plot(pempate, '-', label = 'Empate')
plt.plot(pjog2, '-', label = p2.nome)
plt.ylim([0,1])
plt.hlines(y = 0.5, xmin = 0, xmax = len(pjog1) )
plt.grid(True)
plt.legend()
plt.show()

In [8]:
# Jogar contra quem?
p = p1
tab = owg()
tab.start(p)

O ganhou
Empate
Empate
O ganhou
